In [ ]:
%reload_ext setup

## Demo

In [ ]:
import asyncio
import os
import logging
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from autogen_agentchat.base import TaskResult

from igent.matching_agent import get_agents

# Suppress autogen_core and httpx INFO logs
logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [ ]:
from igent.tools import read_txt, read_json

# Updated Prompts
a_matcher = await read_txt("../prompts/a_matcher.txt")
a_critic = await read_txt("../prompts/a_critic.txt")
b_matcher = await read_txt("../prompts/b_matcher.txt")
b_critic = await read_txt("../prompts/b_critic.txt")


# Workflow Orchestration
async def run_workflow(
    config_path: str,
    registrations_file: str = "registrations.json",
    offers_file: str = "offers.json",
    incentives_file: str = "incentives.json",
    matches_file: str = "matches.json",
    final_matches_file: str = "final_matches.json"
):
    pair1 = await get_agents(
        config_path=config_path,
        matcher_prompt=a_matcher,
        critic_prompt=a_critic
    )
    
    message1 = (
        f"Match registrations to suppliers:\n"
        f"REGISTRATIONS: ```{await read_json(registrations_file)}```\n"
        f"OFFERS: ```{await read_json(offers_file)}```\n"
        f"On approval, SAVE the output to '{matches_file}' using save_json_tool."
    )
    
    print("Running Pair 1...")
    success1 = False
    async for msg in pair1.run_stream(
        task=[TextMessage(content=message1, source="user")],
        cancellation_token=CancellationToken()
    ):
        if isinstance(msg, TextMessage):
            if msg.source == "user":
                print(f"{msg.source}: {msg.content[:100]}")
            else:
                print(f"{msg.source}: {msg.content}")
        elif isinstance(msg, TaskResult):
            final_message = "Pair 1 completed. "
            if msg.stop_reason:
                final_message += f"Stop reason: {msg.stop_reason}"
                if "APPROVE" in msg.stop_reason:
                    success1 = True
            print(final_message)
    
    if not success1:
        print("Pair 1 failed. Aborting.")
        return
    
    # Wait briefly to ensure file write completes
    await asyncio.sleep(0.1)  # Small delay to allow file system sync
    if not os.path.exists(matches_file):
        print(f"Error: '{matches_file}' was not saved. Aborting.")
        return

    # Pair 2: Subsidy Matcher + Subsidy Critic
    pair2 = await get_agents(
        config_path=config_path,
        matcher_prompt=b_matcher,
        critic_prompt=b_critic
    )
    
    message2 = (
        f"Enrich matches with pricing and subsidies:\n"
        f"MATCHES: ```{await read_json(matches_file)}```\n"
        f"OFFERS: ```{await read_json(offers_file)}```\n"
        f"INCENTIVES: ```{await read_json(incentives_file)}```\n"
        f"On approval, SAVE the output to '{final_matches_file}' using save_json_tool."
    )
    
    print("\nRunning Pair 2...")
    async for msg in pair2.run_stream(
        task=[TextMessage(content=message2, source="user")],
        cancellation_token=CancellationToken()
    ):
        if isinstance(msg, TextMessage):
            if msg.source == "user":
                print(f"{msg.source}: {msg.content[:100]}")
            else:
                print(f"{msg.source}: {msg.content}")
        elif isinstance(msg, TaskResult):
            final_message = "Pair 2 completed. "
            if msg.stop_reason:
                final_message += f"Stop reason: {msg.stop_reason}"
            print(final_message)

# Run the workflow
await run_workflow(
    config_path="../static/model_config.yaml",
    registrations_file="../data/demo/registration.json",
    offers_file="../data/demo/offers.json",
    incentives_file="../data/demo/incentives.json",
    matches_file="../data/demo/matches.json",
    final_matches_file="../data/demo/pos.json"
)

## Sandbox

In [ ]:
%reload_ext setup


In [ ]:
from igent.tools import pop_json

await pop_json(source_file="../data/demo/registrations.json", popped_file="../data/demo/popped.json")